# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [81]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

testPicture = load_img('../deerNight.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
# print (pictureArray.shape)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)

model = VGG19()
model.trainable = False
model.summary()

# predict the probability across all output classes
yhat = model.predict(pictureArray)
# ynew = completeModel.predict_classes(pictureArray)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = labels[0][0]
# print the classification
print(labels)


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

2023-09-26 15:19:45.439408: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 330ms/step
[[('n02395406', 'hog', 0.14550796), ('n02797295', 'barrow', 0.040324006), ('n03393912', 'freight_car', 0.024490995), ('n02437616', 'llama', 0.02321001), ('n02793495', 'barn', 0.021253262)]]


## Try with a different input shape

In [82]:
# testPicture = load_img('../deerdownload.jpg', target_size=(2592, 4608))
# testPicture = load_img('../DeerNight.jpg')
testPicture = load_img('../noDeerDay.JPG', target_size=(224,224))
pictureArray = img_to_array(testPicture)
# print (pictureArray.shape)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)



# load model and specify a new input shape for images
new_input = keras.Input(shape=(2592, 4608, 3))
newModel = VGG19(include_top=False, input_tensor=new_input, pooling="max")
newModel.trainable = False
newModel.summary()



Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 2592, 4608, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 2592, 4608, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 2592, 4608, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 1296, 2304, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, 1296, 2304, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 1296, 2304, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 648, 1152, 128)    0     

## Create new Output layer

In [115]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(model)
# Flatten if using newModel
# completeModel.add(Flatten())
# completeModel.add(Dense(200, activation = "relu"))
completeModel.add(Dense(100, activation = "relu"))
completeModel.add(Dense(2, activation = "softmax"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense_35 (Dense)            (None, 100)               100100    
                                                                 
 dense_36 (Dense)            (None, 2)                 202       
                                                                 
Total params: 143,767,542
Trainable params: 100,302
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data

In [116]:
trainImages = keras.utils.image_dataset_from_directory(
    directory='trainingData',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))

Found 2841 files belonging to 2 classes.
Found 316 files belonging to 2 classes.


## Prediction before training

In [117]:
testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
# testPicture = load_img('../DeerNight.jpg', target_size=(224,224))
# testPicture = load_img('../noDeerDay.JPG', target_size=(224,224))
# testPicture = load_img('../city.jpeg', target_size=(224,224))

pictureArray = img_to_array(testPicture)
# reshape data for the model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# load the model

# predict the probability across all output classes
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-09-26 16:07:23.425151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 346ms/step
[[0.49757993 0.50242007]]


## Train Last two layers

In [118]:
completeModel.fit(trainImages, epochs=5, batch_size=10)

Epoch 1/5


2023-09-26 16:07:31.659108: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


89/89 [==============================] - 34s 368ms/step - loss: 0.6228 - accuracy: 0.6758
Epoch 2/5
89/89 [==============================] - 33s 367ms/step - loss: 0.4811 - accuracy: 0.7670
Epoch 3/5
89/89 [==============================] - 33s 366ms/step - loss: 0.3747 - accuracy: 0.8705
Epoch 4/5
89/89 [==============================] - 33s 367ms/step - loss: 0.3208 - accuracy: 0.8888
Epoch 5/5
89/89 [==============================] - 33s 368ms/step - loss: 0.2870 - accuracy: 0.8962


## Test Some data

In [119]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

2023-09-26 16:10:30.525423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 5s 356ms/step - loss: 0.9580 - accuracy: 0.4937
loss =  0.9580318927764893
accuracy =  0.49367091059684753


## See what happens with a deer image

In [141]:
# testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
# testPicture = load_img('../DeerNight.jpg', target_size=(224,224))
# testPicture = load_img('../noDeerDay.JPG', target_size=(224,224))
# testPicture = load_img('../city.jpeg', target_size=(224,224))
testPicture = load_img('./testData/NotDeerTest/467.jpg', target_size=(224,224))
# testPicture = load_img('./testData/deerTest/1749.jpg', target_size=(224,224))

pictureArray = img_to_array(testPicture)
# reshape data for the model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# load the model

# predict the probability across all output classes
yhat = completeModel.predict(pictureArray)
# ynew = completeModel.predict_classes(pictureArray)
# convert the probabilities to class labels
# labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
# label = labels[0][0]
# print the classification
print(yhat)

# completeModel.save("alphaModel.keras")



# 0 is deer 1 is no deer, i need to figure out how to change this

1/1 [==============================] - 0s 25ms/step
[[0.0162286 0.9837714]]
